In [137]:
import base64
import os
from pdf2image import convert_from_path
from openai import OpenAI
import glob
import re
import json

In [3]:

def convert_pdf_to_images(pdf_path, output_folder):
    # Create the output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Convert PDF to images
    images = convert_from_path(pdf_path)

    # Save each image
    for i, image in enumerate(images):
        image.save(os.path.join(output_folder, f'page_{i+1}.png'), 'PNG')



In [9]:

pdf_path = 'minutes/mn20240618.pdf'
output_folder = 'images/mn20240618/pngs'

convert_pdf_to_images(pdf_path, output_folder)


In [10]:

client = OpenAI(
  api_key=key, 
)

def grab_markdown_from_image(image_path):

    with open(image_path, "rb") as image_file:
        encoded_string = base64.b64encode(image_file.read()).decode('utf-8')
    
    mime_type = "image/png"
    
    # Format as a data URI
    data_uri = f"data:{mime_type};base64,{encoded_string}"
    

    response = client.chat.completions.create(
      model="gpt-4o-mini",
      messages=[
        {
          "role": "system",
          "content": [
            {
              "type": "text",
              "text": "You are a highly accurate transcriber. Given an image, your task is to convert the text in the image into its correct Markdown representation. You will only output valid Markdown, ensuring that the transcription is precise. Do not produce code or any other format—only the accurate Markdown representation of the text."
            }
          ]
        },
        {
          "role": "user",
          "content": [
            {
              "type": "image_url",
              "image_url": {
                "url": data_uri
              }
            },
            {
              "type": "text",
              "text": "Convert the following to standard markdown. "
            }
          ]
        }
      ],
      temperature=0.5,
      max_tokens=2048,
      top_p=1,
      frequency_penalty=0,
      presence_penalty=0,
      response_format={
        "type": "text"
      }
    )
    return response
    
image_path = "images/mn20240618/pngs/page_2.png"
output = grab_markdown_from_image(image_path)
print(output.choices[0].message.content)

```markdown
June 18, 2024  
Minutes  

### 20240618 00:07:07  

**A. APPROVAL OF MINUTES**  

May 21, 2024  

**PUBLIC TESTIMONY – None**  

### 20240618 00:08:04  

**MOTION:** (MEYER / KATAYAMA)  
To approve minutes with non-substantive edits.  
**UNANIMOUSLY APPROVED**  
(MIIKE/MEYER/KATAYAMA/CHANG/HANNAHS/KAGAWA-VIVIANI/HO)  

### 20240618 00:09:22  

**B. ACTION ITEM**  

1. Determination and Approval of Draft Agenda by the Commission on Water Resource Management Under Section 92-3.1, HRS, for the Commission to Conduct a Site Visit to Privately-owned Lands to View the Punaluʻu Stream Restoration Project Kamehameha Schools, Punaluʻu Stream, Punaluʻu, Oʻahu, Hawaiʻi on June 27, 2024, as a Limited Meeting, due to the Impracticability of Public Attendance for Reasons of Danger to Health or Safety, or in the Alternative, that On-Site Inspection is Necessary for Future Stream Management Purposes  

**PRESENTATION OF SUBMITTAL:** Deputy Dean Uyen  

Before it may conduct a site visit as 

In [20]:
text = output.choices[0].message.content
data = text.split("\n")

In [24]:
nb_images = len(glob.glob("images/mn20240618/pngs/*.png"))
for image_id in range(nb_images):
    output = grab_markdown_from_image(f"images/mn20240618/pngs/page_{image_id+1}.png")
    with open(f"images/mn20240618/markdown/page_{image_id+1}.md", 'w') as out_file:
        out_file.write(output.choices[0].message.content)
    

In [253]:
def process_mardown(text):
    # remove ``` form the absolute end and remove ```Markdown from absolute beginning `
    new_text = text[12:-4]
    # remove ```Markdown from absolute beginning `
    #new_text = new_text[11:]

    return new_text

In [254]:
nb_images = len(glob.glob("images/mn20240618/pngs/*.png"))
global_ouptut = "\n"
for image_id in range(nb_images):
    md_text = open(f"images/mn20240618/markdown/page_{image_id+1}.md").read()
    md_text_processed = process_mardown(md_text)
    if not md_text[0].isalnum():
        global_ouptut = global_ouptut[:-1]
    
    global_ouptut +=  md_text_processed
    

In [255]:
print(global_ouptut)

# MINUTES
## FOR THE MEETING OF
## THE COMMISSION ON WATER RESOURCE MANAGEMENT

**DATE:** June 18, 2024  
**TIME:** 9:00 a.m.  
**PLACE:** DLNR Boardroom, Kalanimoku Bldg.  
1151 Punchbowl Street, 1<sup>st</sup> Floor  
Online via Zoom, Meeting ID: 883 7811 5081  

Online link to the video recording of the June 18, 2024 Commission on Water Resource Management meeting: [https://vimeo.com/962656881](https://vimeo.com/962656881)

Chairperson Dawn Chang called the meeting of the Commission on Water Resource Management to order at 9:00 a.m. and stated it is a hybrid meeting being held in the Kalanimoku Building boardroom, remotely via Zoom and live streamed via YouTube. It was noted that people may testify via the information provided online. Chairperson Chang reminded the public not to use the chat feature for any comments, as it presents a Sunshine Law issue. She also read the standard contested case statement, took a roll call of Commissioners, and introduced Commission staff.

The follo

In [256]:


def find_all_pattern_line_positions(text, pattern):
    pattern = pattern.upper()
    full_pattern = f'^.*{re.escape(pattern)}.*$'
    
    # Find all occurrences of the pattern in multiline mode
    # We use re.MULTILINE but not re.IGNORECASE
    matches = re.finditer(full_pattern, text, re.MULTILINE)
    
    # List to store all occurrences
    occurrences = []
    
    for match in matches:
        # Get the start and end positions of the matched line
        start_pos = match.start()
        end_pos = match.end()
        occurrences.append((start_pos, end_pos))
    
    return occurrences

In [257]:
sections = ["Action item", "Approval of Minutes"]

In [258]:
all_positions = [] 
for sec in sections:
    locs = find_all_pattern_line_positions(global_ouptut, sec)
    all_positions += locs 
    
all_positions

[(2048, 2064),
 (4795, 4813),
 (61452, 61495),
 (94857, 94902),
 (99530, 99575),
 (1780, 1804)]

In [259]:
def generate_text_chunks(full_text, positions):
    
    # Sorting the list by the first element of each tuple
    sorted_positions = sorted(positions, key=lambda x: x[0])

    chunks = []
    last_start = 0

    # Generate chunks
    for i, (start, end) in enumerate(sorted_positions):
        # Add chunk from last start to current start
        chunks.append(full_text[last_start:start])
        
        # Update last_start for the next iteration
        last_start = start

        # If it's the last tuple, add the final chunk
        if i == len(sorted_positions) - 1:
            chunks.append(full_text[start:])

    return chunks

chunks = generate_text_chunks(global_ouptut, all_positions)

In [275]:
result = {}

for chunk_id, chunk in enumerate(chunks):
    lines = [x for x in chunk.split("\n") if len(x)]  
    mapping = {}
    new_lines = []

    first_line = lines[0].strip()

    for line in lines:
        line = line.strip()
        words = line.split()

        if len(words) > 40:
            new_line = " ".join(words[:15]) + " ___...___ " + " ".join(words[-10:])
            mapping[new_line] = line
            new_lines.append(new_line)
        else:
            new_lines.append(line)

    joined_text = "\n".join(new_lines)

    result[f"chunk_{chunk_id + 1}"] = {
        "section": first_line,
        "original_text": chunk.strip(),    
        "truncated_text": joined_text,        
        "mapping": mapping                  
    }

result_json = json.dumps(result, indent=4)

print(result_json)

{
    "chunk_1": {
        "section": "# MINUTES",
        "original_text": "# MINUTES\n## FOR THE MEETING OF\n## THE COMMISSION ON WATER RESOURCE MANAGEMENT\n\n**DATE:** June 18, 2024  \n**TIME:** 9:00 a.m.  \n**PLACE:** DLNR Boardroom, Kalanimoku Bldg.  \n1151 Punchbowl Street, 1<sup>st</sup> Floor  \nOnline via Zoom, Meeting ID: 883 7811 5081  \n\nOnline link to the video recording of the June 18, 2024 Commission on Water Resource Management meeting: [https://vimeo.com/962656881](https://vimeo.com/962656881)\n\nChairperson Dawn Chang called the meeting of the Commission on Water Resource Management to order at 9:00 a.m. and stated it is a hybrid meeting being held in the Kalanimoku Building boardroom, remotely via Zoom and live streamed via YouTube. It was noted that people may testify via the information provided online. Chairperson Chang reminded the public not to use the chat feature for any comments, as it presents a Sunshine Law issue. She also read the standard contested case 

In [276]:
# Empty Schema with expected types
full_schema = {
    "meeting_info": {
        "title": "string",
        "date": "string",
        "time": "string",
        "location": {
            "place": "string",
            "online_option": True
        }
    },
    "attendance": {
        "members": [
            {"name": "string", "role": "string"},
        ],
        "counsel": [
            {"name": "string", "role": "string"},
        ],
        "staff": [
            {"name": "string", "role": "string"},
        ],
        "others": [
            {"name": "string", "affiliation": "string"},
        ]
    },
    "spontaneous_requests": [
        {
            "request": "string",
            "motion": {
                "motion_proposal": "string",
                "proposed_by": "string",
                "seconded_by": "string",
                "action": "string",
                "votes": {
                    "for": ["string"],
                    "against": ["string"],
                    "abstain": ["string"],
                    "absent": ["string"]
                }
            }
        }
    ],
    "approval_of_minutes": {
        "section": "string",
        "time_stamp": "string",
        "questions_comments": [
            {"name":"string",
              "statement":"string"},
              ],
        "public_testimony": [
            {"name":"string",
              "statement":"string"},
              ],
        "motion": {
            "motion_proposal": "string",
            "proposed_by": "string",
            "seconded_by": "string",
            "action": "string",
            "votes": {
                "for": ["string"],
                "against": ["string"],
                "abstain": ["string"],
                "absent": ["string"]
            }
        }
    },
    "non_action_items": {
        "section": "string",
        "time_stamp": "string",
        "submittals": [
            {
                "submittal": "string",
                "title": "string",
                "timestamp": "string",
                "presentations": [
                    {
                        "presenter": {
                            "name": "string",
                            "affiliation": "string"
                        },
                        "external_resource": False,
                        "external_resource_location": "string",
                        "comments": "string",
                        "presentation": "string"
                    }
                ],
                "questions_comments": [
                    {"name":"string",
                      "statement":"string"},
                ],
                "public_testimony": [
                    {"name":"string",
                      "statement":"string"},
                ]
            }
        ]
    },
    "action_items": {
        "section": "string",
        "timestamp": "string",
        "submittals": [
            {
                "submittal": "string",
                "title": "string",
                "timestamp": "string",
                "presentations": [
                    {
                        "presenter": {
                            "name": "string",
                            "affiliation": "string"
                        },
                        "external_resource": False,
                        "external_resource_location": "string",
                        "comments": "string",
                        "presentation": "string"
                    }
                ],
                "questions_comments": [                    
                    {"name":"string",
                      "statement":"string"},
                ],
                "public_testimony": [
                    {"name":"string",
                      "statement":"string"},
                ],
                "discussion": [                    
                    {"name":"string",
                      "statement":"string"},
                ],
                "motion": {
                    "motion_proposal": "string",
                    "proposed_by": "string",
                    "seconded_by": "string",
                    "action": "string",
                    "votes": {
                        "for": ["string"],
                        "against": ["string"],
                        "abstain": ["string"],
                        "absent": ["string"]
                    }
                }
            }
        ]
    }
}

# Empty Schema 
empty_schema = {
    "meeting_info": {
        "title": "",
        "date": "",
        "time": "",
        "location": {
            "place": "",
            "online_option": False
        }
    },
    "attendance": {
        "members": [
            {"name": "", "role": ""},
        ],
        "counsel": [
            {"name": "", "role": ""},
        ],
        "staff": [
            {"name": "", "role": ""},
        ],
        "others": [
            {"name": "", "affiliation": ""},
        ]
    },
    "spontaneous_requests": [
        {
            "request": "",
            "motion": {
                "motion_proposal": "",
                "proposed_by": "",
                "seconded_by": "",
                "action": "",
                "votes": {
                    "for": [""],
                    "against": [""],
                    "abstain": [""],
                    "absent": [""]
                }
            }
        }
    ],
    "approval_of_minutes": {
        "section": "",
        "time_stamp": "",
        "questions_comments": [
            {"name": "",
             "statement": ""},
        ],
        "public_testimony": [
            {"name": "",
             "statement": ""},
        ],
        "motion": {
            "motion_proposal": "",
            "proposed_by": "",
            "seconded_by": "",
            "action": "",
            "votes": {
                "for": [""],
                "against": [""],
                "abstain": [""],
                "absent": [""]
            }
        }
    },
    "non_action_items": {
        "section": "",
        "time_stamp": "",
        "submittals": [
            {
                "submittal": "",
                "title": "",
                "timestamp": "",
                "presentations": [
                    {
                        "presenter": {
                            "name": "",
                            "affiliation": ""
                        },
                        "external_resource": False,
                        "external_resource_location": "",
                        "comments": "",
                        "presentation": ""
                    }
                ],
                "questions_comments": [
                    {"name": "",
                     "statement": ""},
                ],
                "public_testimony": [
                    {"name": "",
                     "statement": ""},
                ]
            }
        ]
    },
    "action_items": {
        "section": "",
        "timestamp": "",
        "submittals": [
            {
                "submittal": "",
                "title": "",
                "timestamp": "",
                "presentations": [
                    {
                        "presenter": {
                            "name": "",
                            "affiliation": ""
                        },
                        "external_resource": False,
                        "external_resource_location": "",
                        "comments": "",
                        "presentation": ""
                    }
                ],
                "questions_comments": [
                    {"name": "",
                     "statement": ""},
                ],
                "public_testimony": [
                    {"name": "",
                     "statement": ""},
                ],
                "discussion": [
                    {"name": "",
                     "statement": ""},
                ],
                "motion": {
                    "motion_proposal": "",
                    "proposed_by": "",
                    "seconded_by": "",
                    "action": "",
                    "votes": {
                        "for": [""],
                        "against": [""],
                        "abstain": [""],
                        "absent": [""]
                    }
                }
            }
        ]
    }
}

# Schema example using minutes from April 18,2023
meeting = {
  "meeting_info": {
      "title": "The Commission on Water Resource Management",
      "date": "April 18, 2023",
      "time": "9:00 AM",
      "location": {
        "place": "Kalanimoku Building boardroom",
        "online_option": True
      }
    },
  "attendance": {
      "members": [
        { "name": "Dawn Chang", "role": "Chairperson" },
        { "name": "Mr. Michael Buck", "role": "Member" },
        { "name": "Mr. Neil Hannahs", "role": "Member" },
        { "name": "Dr. Aurora Kagawa-Viviani", "role": "Member" },
        { "name": "Mr. Wayne Katayama", "role": "Member" },
        { "name": "Mr. Paul Meyer", "role": "Member" }
      ],
      "counsel": [
        { "name": "Ms. Kathleen Ho", "role": "Counsel" }
      ],
      "staff": [
        { "name": "M. Kaleo Manuel", "role": "Deputy" },
        { "name": "Ms. Nadine Pomroy", "role": "Staff" },
        { "name": "Mr. Ayron Strauch", "role": "Staff" },
        { "name": "Mr. Neal Fujii", "role": "Staff" },
        { "name": "Mr. Dean Uyeno", "role": "Staff" },
        { "name": "Ms. Katie Roth", "role": "Staff" },
        { "name": "Mr. Ryan Imata", "role": "Staff" },
        { "name": "Ms. Alexa Deike", "role": "Staff" },
        { "name": "Mr. Barrett Won", "role": "Staff" }
      ],
      "others": [
        { "name": "Dr. Ryan Longman", "affiliation": "University of Hawai'i" },
        { "name": "Maj General Mark Hashimoto", "affiliation": "U.S. Marine Corps" },
        { "name": "Colonel Kevin Williams", "affiliation": "Chief of Staff for the JTF Red Hill" },
        { "name": "Sarah Moody", "affiliation": "Naval Facilities Engineering Command Hawai'i" },
        { "name": "Captain Cameron Geertsma", "affiliation": "Commanding Officer for Naval facilities Engineering assistance command Hawai‘I" },
        { "name": "Kevin Nakamura", "affiliation": "Chief Medical Officer for the defense health agency region Indo-Pacific" },
        { "name": "Mark Vaught", "affiliation": "Mahi Pono" },
        { "name": "Cal Chipchase", "affiliation": "Molokai Ranch" },
        { "name": "Harold Edwards", "affiliation": "ITC Consultant/Contractor" }
      ]
    },
  "spontaneous_requests": [
      {
        "request": "Chair Chang requested a motion to take C1 and C2 out of the agenda order.",
        "motion": { 
          "motion_proposal": "Take item C1 and item C2 and move up.",
          "proposed_by": "HO",
          "seconded_by": "Meyer",
          "action": "Unanimously Approved",
          "votes": {
            "for": ["HO","BUCK","HANNAHS","KAGAWA-VIVIANI","KATAYAMA","MEYER","CHANG"],
            "against": [],
            "abstain": [],
            "absent": []
          }
        }
      },
    ],
  "approval_of_minutes": {
      "section": "A",
      "time_stamp": "00:08:10",
      "questions_comments": [],
      "public_testimony": [],
      "motion": {
        "motion_proposal": "To approve the March 21, 2023, minutes subject to the proposed amendments by Commissioner Kagawa-Viviani",
        "proposed_by": "BUCK",
        "seconded_by": "HO",
        "action": "Unanimously Approved",
        "votes": {
          "for": ["HO","BUCK","HANNAHS","KAGAWA-VIVIANI","KATAYAMA","MEYER","CHANG"],
          "against": [],
          "abstain": [],
          "absent": []
        }
      },
    },
  "non_action_items":{
      "section": "C",
      "time_stamp": "00:10:37",
      "submittals": [
        {
        "submittal": "C1",
        "title": "Informational Briefing on the Pacific Drought Knowledge Exchange by Dr. Ryan Longman, University of Hawai‘i",
        "timestamp": "00:10:37",
        "presentations": [
            {
              "presenter": { "name": "Dr. Ryan Longman", "affiliation": "University of Hawai‘i" },
              "external_resource":True,
              "external_resource_location":"",
              "comments":"Dr. Ryan Longman provided a slide presentation on an overview of The Pacific Drought Knowledge Exchange (PDKE). It was framed within the context..... (Slide presentation attached.)",
              "presentation":"",
            }
          ],
        "questions_comments": [
            {"name":"Commissioner Ho",
              "statement":"are you coordinating or is the school of public health interested in your research and the effects climate change has on public health and the environment?"
            },
            {"name":"Dr. Longman",
              "statement":"it’s an open research question. I’m sure there’s an interest in that. .....explore and certainly something that they might be able to utilize."
            },
          ],
        "public_testimony": [
            {"name":"",
              "statement":""
            },
            {"name":"",
              "statement":""
            },
          ],
        },
        {
        "submittal": "C2",
        "title": "Joint Task Force Red Hill, Navy, and Defense Health Agency Overviews/Updates of Defueling, Site Remediation and Closure, Red Hill Shaft Recovery and Monitoring, and Medical Support to Families",
        "time_stamp": "01:11:37",
        "presentations": [
            {
              "presenter": { "name": "Maj General Mark Hashimoto", "affiliation": "U.S Marine Corps" },
              "external_resource":False,
              "external_resource_location":"",
              "comments":"",
              "presentation":"Thanks everyone for the opportunity to provide updates from organizations involved in the defueling......Each organization will present a brief update on what....",
            },
            {
              "presenter": { "name": "Colonel Kevin Williams", "affiliation": "the Chief of Staff for the JTF Red Hill" },
              "external_resource":True,
              "external_resource_location":"...",
              "comments":"Colonel Kevin Williams, the chief of staff for the Joint Task Force Red Hill, thanks everyone....He works for Admiral Wade and will discuss their mission and timeline while also answering any questions. (See PowerPoint presentation.)",
              "presentation":"",
            },
            {
              "presenter": { "name": "Maj General Mark Hashimoto", "affiliation": "U.S Marine Corps" },
              "external_resource":False,
              "external_resource_location":"",
              "comments":"",
              "presentation":"",
            },
          ],
          "questions_comments": [
            {
              "name": "Commissioner Hannahs",
              "statement": "thank the presenters for their excellent presentation and I note four fundamental commitments: to mitigate or remediate..."
            },
            {
              "name": "Captain Cameron Geertsma",
              "statement": "Yes sir."
            },
          ],
          "public_testimony": [
            {
              "name": "Katherine McClanahan",
              "statement": "The testifier restated their written testimony.",
              "written_testimony":"..."
            },
            {
              "name": "Healani Pale-Sonoda",
              "statement": "The testifier restated their written testimony..",
              "written_testimony":"..."
            },
            {
              "name": "Susan Pcola-Davis",
              "statement": "Sent in a written testimony but has additional concerns. • Condensation is being removed from fuel tanks this week to check fuel quality, and if it is low, the fuel can’t be reused. • She asks if dewatering needs to be done prior to defueling again since condensation builds up. ...",
              "written_testimony":""
            }
          ],
        }
      ]
    },
  "action_items":{
      "section": "B",
      "timestamp": "03:06:10",
      "submittals": [
        {
        "submittal": "B1",
        "title": "Approval of Stream Diversion Works Permit Application (SDWP.5951.6) and Special Conditions to East Maui Irrigation, Company, LLC, for Abandonment of Stream Diversion Works No. 184.6, Allowing Applicant to Breach and Remove the Kapala‘alaea Dam, Reseed, and Add Erosion Protection; Papalua (Piiloi) Stream, Ha‘ikū, Maui, Tax Map Key: (2) 2-8-007:001.",
        "timestamp": "03:06:10",
        "presentations": [
            {
              "presenter": { "name": "Deputy Kaleo Manuel", "affiliation": "CWRM Branch" },
              "external_resource":False,
              "external_resource_location":"",
              "comments":"Deputy Manuel read the summary of request as submitted and stated that staff stands on its submittal. Deputy Manuel then read the staff’s recommendations.",
              "presentation":"",
            }
          ],
        "questions_comments": [
            {"name":"Commissioner Buck",
              "statement":"Kaleo, seems like the theme for this meeting. Obviously, our East Maui decision was abandoning a lot of unnecessary diversions, yet..."
            },
            {"name":"Deputy Manuel",
              "statement":"it’s an open research question. I’m sure there’s an interest in that. I’m not particularly working with anybody in public health but it’s certainly an area to explore and certainly something that they might be able to utilize."
            },
          ],
        "public_testimony": [
            {"name":"Lucienne Denaie",
              "statement":"Chairperson of the Sierra Club Maui Group. • The streams affected by the dam are not part of the East Maui decision and are not regulated by IIFS ..."
            },
            {"name":"Jason Kent",
              "statement":"Consultant for East Maui Irrigation. • He clarified that an environmental assessment is not required for the project since Nationwide permits number 3 and 13 will be applicable. ..."
            },
          ],
        "discussion": [
            {"name":"Commissioner Hannahs",
              "statement":"regarding local consultation with ‘Aha Moku Council as well as immediate residence. Is it possible to add them..."
            },
            {"name":"Deputy Manuel",
              "statement":"based on the conversation and having to wait for SHPD and because this is also tied to a jurisdiction of DLNR, I'm comfortable..."
            },
          ],
        "motion": {
            "motion_proposal": "To defer item B-1 to allow applicant time to confer with Aha Moku Council and other affected parties so that the Commission can do a proper Ka Pa‘akai analysis.",
            "proposed_by": "HANNAS",
            "seconded_by": "BUCK",
            "action": "Unanimously Approved",
            "votes": {
              "for": ["HO","BUCK","HANNAHS","KAGAWA-VIVIANI","KATAYAMA","MEYER","CHANG"],
              "against": [],
              "abstain": [],
              "absent": []
            }
          },
        },
        {
        "submittal": "B2",
        "title": "Approval of Stream Diversion Works Permit Application (SDWP.5970.4) and Special Conditions, Molokai Properties Limited, Abandonment of Stream Diversion Works No. 862.4, to Remove Pipes and Concrete from Stream by Use of Hand Tools, West Kawela Stream, Kawela, Moloka‘i, Tax Map Key: (2) 5-4- 003:026; and Declare that Project is Exempt from Environmental Assessment Requirements under Hawaii Revised Statutes Chapter 343, and Hawaii Administrative Rules Chapter 11-200.1",
        "timestamp": "04:17:00",
        "presentations": [
            {
              "presenter": { "name": "Deputy Kaleo Manuel", "affiliation": "CWRM Branch" },
              "external_resource":False,
              "external_resource_location":"",
              "comments":"Deputy Manuel stated the summary of the briefing item to include some recommendations and amendments involving work activities that may impact the environment, particularly water birds...He asks that the same recommendation is made for item B-3 as well",
              "presentation":"",
            }
          ],
        "questions_comments": [
            {"name":"Commissioner Buck",
              "statement":"are we removing the abandoned pipes and concrete from the stream channel?"
            },
            {"name":"Deputy Manuel",
              "statement":"yes."
            },
          ],
        "public_testimony": [
            {"name":"Teave Heave",
              "statement":"She is a born and raised resident of Kawela and a member of Molokai No Ka Heke. • She expresses her support for the action to rectify the mess caused by old pipes and diversion materials in the area. ..."
            },
            {"name":"Mahesh Cleveland with Earth Justice",
              "statement":"• An attorney from Earth Justice representing an organization that filed a petition in 2019 to amend flow standards for the central Moloka‘i streams. • He asks that the ranch be compelled to go through the formal abandonment processes for the specific purpose of cleaning ..."
            },
          ],
        "discussion": [
            {"name":"Commissioner Hannahs",
              "statement":"thank the community and the applicant for testifying and taking action to close the diversions down. Removing ‘ōpala and returning ..."
            },
            {"name":"Chair Chang",
              "statement":"I guess I would take that one step further. I don't know if we even need a plan. My fear..."
            },
          ],
        "motion": {
            "motion_proposal": "To adopt the recommendation of staff with the amendment that the removal of pipes and concrete be in accordance with a plan submitted by MPL and approved by commission staff within 6 months.",
            "proposed_by": "HANNAS",
            "seconded_by": "BUCK",
            "action": "Unanimously Approved",
            "votes": {
              "for": ["HO","BUCK","HANNAHS","KAGAWA-VIVIANI","KATAYAMA","MEYER","CHANG"],
              "against": [],
              "abstain": [],
              "absent": []
            }
          },
        },
        {
        "submittal": "B3",
        "title": "Approval of Stream Diversion Works Permit Application (SDWP.5971.4) and Special Conditions, Molokai Properties Limited, Abandonment of Stream Diversion Works No. 866.4, to Remove Pipes and Concrete from Stream Intake by Use of Hand Tools, Unnamed Tributary to East Kawela Stream, Kawela, Moloka‘i, Tax Map Key: (2) 5- 4-003:026; and Declare that Project is Exempt from Environmental Assessment Requirements under Hawaii Revised Statutes Chapter 343, and Hawaii Administrative Rules Chapter 11- 200.1",
        "timestamp": "05:09:38",
        "presentations": [
            {
              "presenter": { "name": "", "affiliation": "" },
              "external_resource":False,
              "external_resource_location":"",
              "comments":"",
              "presentation":"",
            }
          ],
        "questions_comments": [
            {"name":"Commissioner Hannahs",
              "statement":"Kaleo, seems like the theme for this meeting. Obviously, our East Maui decision was abandoning a lot of unnecessary diversions, yetI would suggest we consider this with the same amendment regarding the..."
            },
            {"name":"Commissioner Kagawa-Viviani",
              "statement":"I did not receive the Earth Justice’s late testimony. I just want it on..."
            },
          ],
        "public_testimony": [
            {"name":"Mahesh Clevelande",
              "statement":" Chapter 343 of HEPA requires an environmental assessment for actions that may cause significant environmental impacts, triggered in part by presence on conservation lands. ..."
            },
            {"name":"Cal Chipchase",
              "statement":"the decision that is being considered by the commission regarding the removal of diversion from a stream, which is evaluated as exempt and appropriate. I believe that the exemption declaration is entirely correct, and that ..."
            },
          ],
        "discussion": [
            {"name":"",
              "statement":""
            },
            {"name":"",
              "statement":""
            },
          ],
        "motion": {
            "motion_proposal": "To adopt the recommendation of staff with the amendment that the removal of pipes and concrete be in accordance with a plan submitted by MPL and approved by commission staff within 6 months.",
            "proposed_by": "HANNAS",
            "seconded_by": "KAGAWA-VIVIANI",
            "action": "Unanimously Approved",
            "votes": {
              "for": ["HO","BUCK","HANNAHS","KAGAWA-VIVIANI","KATAYAMA","MEYER","CHANG"],
              "against": [],
              "abstain": [],
              "absent": []
            }
          },
        },
      ]    
    },
}

# Schema broken down into sections
schemas = [
    {
        "header": "NON-ACTION ITEMS / INFORMATIONAL BRIEFINGS",
        "schema":   {
            "non_action_items":{
            "section": "C",
            "time_stamp": "00:10:37",
            "submittals": [
              {
              "submittal": "C1",
              "title": "Informational Briefing on the Pacific Drought Knowledge Exchange by Dr. Ryan Longman, University of Hawai‘i",
              "timestamp": "00:10:37",
              "presentations": [
                  {
                    "presenter": { "name": "Dr. Ryan Longman", "affiliation": "University of Hawai‘i" },
                    "external_resource":True,
                    "external_resource_location":"",
                    "comments":"Dr. Ryan Longman provided a slide presentation on an overview of The Pacific Drought Knowledge Exchange (PDKE). It was framed within the context..... (Slide presentation attached.)",
                    "presentation":"",
                  }
                ],
              "questions_comments": [
                  {"name":"Commissioner Ho",
                    "statement":"are you coordinating or is the school of public health interested in your research and the effects climate change has on public health and the environment?"
                  },
                  {"name":"Dr. Longman",
                    "statement":"it’s an open research question. I’m sure there’s an interest in that. .....explore and certainly something that they might be able to utilize."
                  },
                ],
              "public_testimony": [
                  {"name":"",
                    "statement":""
                  },
                  {"name":"",
                    "statement":""
                  },
                ],
              },
              {
              "submittal": "C2",
              "title": "Joint Task Force Red Hill, Navy, and Defense Health Agency Overviews/Updates of Defueling, Site Remediation and Closure, Red Hill Shaft Recovery and Monitoring, and Medical Support to Families",
              "time_stamp": "01:11:37",
              "presentations": [
                  {
                    "presenter": { "name": "Maj General Mark Hashimoto", "affiliation": "U.S Marine Corps" },
                    "external_resource":False,
                    "external_resource_location":"",
                    "comments":"",
                    "presentation":"Thanks everyone for the opportunity to provide updates from organizations involved in the defueling......Each organization will present a brief update on what....",
                  },
                  {
                    "presenter": { "name": "Colonel Kevin Williams", "affiliation": "the Chief of Staff for the JTF Red Hill" },
                    "external_resource":True,
                    "external_resource_location":"...",
                    "comments":"Colonel Kevin Williams, the chief of staff for the Joint Task Force Red Hill, thanks everyone....He works for Admiral Wade and will discuss their mission and timeline while also answering any questions. (See PowerPoint presentation.)",
                    "presentation":"",
                  },
                  {
                    "presenter": { "name": "Maj General Mark Hashimoto", "affiliation": "U.S Marine Corps" },
                    "external_resource":False,
                    "external_resource_location":"",
                    "comments":"",
                    "presentation":"",
                  },
                ],
                "questions_comments": [
                  {
                    "name": "Commissioner Hannahs",
                    "statement": "thank the presenters for their excellent presentation and I note four fundamental commitments: to mitigate or remediate..."
                  },
                  {
                    "name": "Captain Cameron Geertsma",
                    "statement": "Yes sir."
                  },
                ],
                "public_testimony": [
                  {
                    "name": "Katherine McClanahan",
                    "statement": "The testifier restated their written testimony.",
                    "written_testimony":"..."
                  },
                  {
                    "name": "Healani Pale-Sonoda",
                    "statement": "The testifier restated their written testimony..",
                    "written_testimony":"..."
                  },
                  {
                    "name": "Susan Pcola-Davis",
                    "statement": "Sent in a written testimony but has additional concerns. • Condensation is being removed from fuel tanks this week to check fuel quality, and if it is low, the fuel can’t be reused. • She asks if dewatering needs to be done prior to defueling again since condensation builds up. ...",
                    "written_testimony":""
                  }
                ],
              }
            ]
          },
          "spontaneous_requests": [
                {
                  "request": "Chair Chang requested a motion to take C1 and C2 out of the agenda order.",
                  "motion": { 
                    "motion_proposal": "Take item C1 and item C2 and move up.",
                    "proposed_by": "HO",
                    "seconded_by": "Meyer",
                    "action": "Unanimously Approved",
                    "votes": {
                      "for": ["HO","BUCK","HANNAHS","KAGAWA-VIVIANI","KATAYAMA","MEYER","CHANG"],
                      "against": [],
                      "abstain": [],
                      "absent": []
                    }
                  }
                },
              ],
        }
    },
    {
        "header": "ACTION ITEMS",
        "schema": {
            "action_items":{
            "section": "B",
            "timestamp": "03:06:10",
            "submittals": [
              {
              "submittal": "B1",
              "title": "Approval of Stream Diversion Works Permit Application (SDWP.5951.6) and Special Conditions to East Maui Irrigation, Company, LLC, for Abandonment of Stream Diversion Works No. 184.6, Allowing Applicant to Breach and Remove the Kapala‘alaea Dam, Reseed, and Add Erosion Protection; Papalua (Piiloi) Stream, Ha‘ikū, Maui, Tax Map Key: (2) 2-8-007:001.",
              "timestamp": "03:06:10",
              "presentations": [
                  {
                    "presenter": { "name": "Deputy Kaleo Manuel", "affiliation": "CWRM Branch" },
                    "external_resource":False,
                    "external_resource_location":"",
                    "comments":"Deputy Manuel read the summary of request as submitted and stated that staff stands on its submittal. Deputy Manuel then read the staff’s recommendations.",
                    "presentation":"",
                  }
                ],
              "questions_comments": [
                  {"name":"Commissioner Buck",
                    "statement":"Kaleo, seems like the theme for this meeting. Obviously, our East Maui decision was abandoning a lot of unnecessary diversions, yet..."
                  },
                  {"name":"Deputy Manuel",
                    "statement":"it’s an open research question. I’m sure there’s an interest in that. I’m not particularly working with anybody in public health but it’s certainly an area to explore and certainly something that they might be able to utilize."
                  },
                ],
              "public_testimony": [
                  {"name":"Lucienne Denaie",
                    "statement":"Chairperson of the Sierra Club Maui Group. • The streams affected by the dam are not part of the East Maui decision and are not regulated by IIFS ..."
                  },
                  {"name":"Jason Kent",
                    "statement":"Consultant for East Maui Irrigation. • He clarified that an environmental assessment is not required for the project since Nationwide permits number 3 and 13 will be applicable. ..."
                  },
                ],
              "discussion": [
                  {"name":"Commissioner Hannahs",
                    "statement":"regarding local consultation with ‘Aha Moku Council as well as immediate residence. Is it possible to add them..."
                  },
                  {"name":"Deputy Manuel",
                    "statement":"based on the conversation and having to wait for SHPD and because this is also tied to a jurisdiction of DLNR, I'm comfortable..."
                  },
                ],
              "motion": {
                  "motion_proposal": "To defer item B-1 to allow applicant time to confer with Aha Moku Council and other affected parties so that the Commission can do a proper Ka Pa‘akai analysis.",
                  "proposed_by": "HANNAS",
                  "seconded_by": "BUCK",
                  "action": "Unanimously Approved",
                  "votes": {
                    "for": ["HO","BUCK","HANNAHS","KAGAWA-VIVIANI","KATAYAMA","MEYER","CHANG"],
                    "against": [],
                    "abstain": [],
                    "absent": []
                  }
                },
              },
              {
              "submittal": "B2",
              "title": "Approval of Stream Diversion Works Permit Application (SDWP.5970.4) and Special Conditions, Molokai Properties Limited, Abandonment of Stream Diversion Works No. 862.4, to Remove Pipes and Concrete from Stream by Use of Hand Tools, West Kawela Stream, Kawela, Moloka‘i, Tax Map Key: (2) 5-4- 003:026; and Declare that Project is Exempt from Environmental Assessment Requirements under Hawaii Revised Statutes Chapter 343, and Hawaii Administrative Rules Chapter 11-200.1",
              "timestamp": "04:17:00",
              "presentations": [
                  {
                    "presenter": { "name": "Deputy Kaleo Manuel", "affiliation": "CWRM Branch" },
                    "external_resource":False,
                    "external_resource_location":"",
                    "comments":"Deputy Manuel stated the summary of the briefing item to include some recommendations and amendments involving work activities that may impact the environment, particularly water birds...He asks that the same recommendation is made for item B-3 as well",
                    "presentation":"",
                  }
                ],
              "questions_comments": [
                  {"name":"Commissioner Buck",
                    "statement":"are we removing the abandoned pipes and concrete from the stream channel?"
                  },
                  {"name":"Deputy Manuel",
                    "statement":"yes."
                  },
                ],
              "public_testimony": [
                  {"name":"Teave Heave",
                    "statement":"She is a born and raised resident of Kawela and a member of Molokai No Ka Heke. • She expresses her support for the action to rectify the mess caused by old pipes and diversion materials in the area. ..."
                  },
                  {"name":"Mahesh Cleveland with Earth Justice",
                    "statement":"• An attorney from Earth Justice representing an organization that filed a petition in 2019 to amend flow standards for the central Moloka‘i streams. • He asks that the ranch be compelled to go through the formal abandonment processes for the specific purpose of cleaning ..."
                  },
                ],
              "discussion": [
                  {"name":"Commissioner Hannahs",
                    "statement":"thank the community and the applicant for testifying and taking action to close the diversions down. Removing ‘ōpala and returning ..."
                  },
                  {"name":"Chair Chang",
                    "statement":"I guess I would take that one step further. I don't know if we even need a plan. My fear..."
                  },
                ],
              "motion": {
                  "motion_proposal": "To adopt the recommendation of staff with the amendment that the removal of pipes and concrete be in accordance with a plan submitted by MPL and approved by commission staff within 6 months.",
                  "proposed_by": "HANNAS",
                  "seconded_by": "BUCK",
                  "action": "Unanimously Approved",
                  "votes": {
                    "for": ["HO","BUCK","HANNAHS","KAGAWA-VIVIANI","KATAYAMA","MEYER","CHANG"],
                    "against": [],
                    "abstain": [],
                    "absent": []
                  }
                },
              },
              {
              "submittal": "B3",
              "title": "Approval of Stream Diversion Works Permit Application (SDWP.5971.4) and Special Conditions, Molokai Properties Limited, Abandonment of Stream Diversion Works No. 866.4, to Remove Pipes and Concrete from Stream Intake by Use of Hand Tools, Unnamed Tributary to East Kawela Stream, Kawela, Moloka‘i, Tax Map Key: (2) 5- 4-003:026; and Declare that Project is Exempt from Environmental Assessment Requirements under Hawaii Revised Statutes Chapter 343, and Hawaii Administrative Rules Chapter 11- 200.1",
              "timestamp": "05:09:38",
              "presentations": [
                  {
                    "presenter": { "name": "", "affiliation": "" },
                    "external_resource":False,
                    "external_resource_location":"",
                    "comments":"",
                    "presentation":"",
                  }
                ],
              "questions_comments": [
                  {"name":"Commissioner Hannahs",
                    "statement":"Kaleo, seems like the theme for this meeting. Obviously, our East Maui decision was abandoning a lot of unnecessary diversions, yetI would suggest we consider this with the same amendment regarding the..."
                  },
                  {"name":"Commissioner Kagawa-Viviani",
                    "statement":"I did not receive the Earth Justice’s late testimony. I just want it on..."
                  },
                ],
              "public_testimony": [
                  {"name":"Mahesh Clevelande",
                    "statement":" Chapter 343 of HEPA requires an environmental assessment for actions that may cause significant environmental impacts, triggered in part by presence on conservation lands. ..."
                  },
                  {"name":"Cal Chipchase",
                    "statement":"the decision that is being considered by the commission regarding the removal of diversion from a stream, which is evaluated as exempt and appropriate. I believe that the exemption declaration is entirely correct, and that ..."
                  },
                ],
              "discussion": [
                  {"name":"",
                    "statement":""
                  },
                  {"name":"",
                    "statement":""
                  },
                ],
              "motion": {
                  "motion_proposal": "To adopt the recommendation of staff with the amendment that the removal of pipes and concrete be in accordance with a plan submitted by MPL and approved by commission staff within 6 months.",
                  "proposed_by": "HANNAS",
                  "seconded_by": "KAGAWA-VIVIANI",
                  "action": "Unanimously Approved",
                  "votes": {
                    "for": ["HO","BUCK","HANNAHS","KAGAWA-VIVIANI","KATAYAMA","MEYER","CHANG"],
                    "against": [],
                    "abstain": [],
                    "absent": []
                  }
                },
              },
            ]    
          },
        },
            "spontaneous_requests": [
                {
                  "request": "Chair Chang requested a motion to take C1 and C2 out of the agenda order.",
                  "motion": { 
                    "motion_proposal": "Take item C1 and item C2 and move up.",
                    "proposed_by": "HO",
                    "seconded_by": "Meyer",
                    "action": "Unanimously Approved",
                    "votes": {
                      "for": ["HO","BUCK","HANNAHS","KAGAWA-VIVIANI","KATAYAMA","MEYER","CHANG"],
                      "against": [],
                      "abstain": [],
                      "absent": []
                    }
                  }
                },
              ],
    },
    {
        "header": "APPROVAL OF MINUTES",
        "schema": {
            "approval_of_minutes": {
                "section": "A",
                "time_stamp": "00:08:10",
                "questions_comments": [],
                "public_testimony": [],
                "motion": {
                  "motion_proposal": "To approve the March 21, 2023, minutes subject to the proposed amendments by Commissioner Kagawa-Viviani",
                  "proposed_by": "BUCK",
                  "seconded_by": "HO",
                  "action": "Unanimously Approved",
                  "votes": {
                    "for": ["HO","BUCK","HANNAHS","KAGAWA-VIVIANI","KATAYAMA","MEYER","CHANG"],
                    "against": [],
                    "abstain": [],
                    "absent": []
                  }
                },
            },
            "spontaneous_requests": [
                {
                  "request": "Chair Chang requested a motion to take C1 and C2 out of the agenda order.",
                  "motion": { 
                    "motion_proposal": "Take item C1 and item C2 and move up.",
                    "proposed_by": "HO",
                    "seconded_by": "Meyer",
                    "action": "Unanimously Approved",
                    "votes": {
                      "for": ["HO","BUCK","HANNAHS","KAGAWA-VIVIANI","KATAYAMA","MEYER","CHANG"],
                      "against": [],
                      "abstain": [],
                      "absent": []
                    }
                  }
                },
              ],
        }
    },
    {
        "header": "INTRO",
        "schema": {
            "meeting_info": {
                "title": "The Commission on Water Resource Management",
                "date": "April 18, 2023",
                "time": "9:00 AM",
                "location": {
                  "place": "Kalanimoku Building boardroom",
                  "online_option": True
                }
              },
            "attendance": {
                "members": [
                  { "name": "Dawn Chang", "role": "Chairperson" },
                  { "name": "Mr. Michael Buck", "role": "Member" },
                  { "name": "Mr. Neil Hannahs", "role": "Member" },
                  { "name": "Dr. Aurora Kagawa-Viviani", "role": "Member" },
                  { "name": "Mr. Wayne Katayama", "role": "Member" },
                  { "name": "Mr. Paul Meyer", "role": "Member" }
                ],
                "counsel": [
                  { "name": "Ms. Kathleen Ho", "role": "Counsel" }
                ],
                "staff": [
                  { "name": "M. Kaleo Manuel", "role": "Deputy" },
                  { "name": "Ms. Nadine Pomroy", "role": "Staff" },
                  { "name": "Mr. Ayron Strauch", "role": "Staff" },
                  { "name": "Mr. Neal Fujii", "role": "Staff" },
                  { "name": "Mr. Dean Uyeno", "role": "Staff" },
                  { "name": "Ms. Katie Roth", "role": "Staff" },
                  { "name": "Mr. Ryan Imata", "role": "Staff" },
                  { "name": "Ms. Alexa Deike", "role": "Staff" },
                  { "name": "Mr. Barrett Won", "role": "Staff" }
                ],
                "others": [
                  { "name": "Dr. Ryan Longman", "affiliation": "University of Hawai'i" },
                  { "name": "Maj General Mark Hashimoto", "affiliation": "U.S. Marine Corps" },
                  { "name": "Colonel Kevin Williams", "affiliation": "Chief of Staff for the JTF Red Hill" },
                  { "name": "Sarah Moody", "affiliation": "Naval Facilities Engineering Command Hawai'i" },
                  { "name": "Captain Cameron Geertsma", "affiliation": "Commanding Officer for Naval facilities Engineering assistance command Hawai‘I" },
                  { "name": "Kevin Nakamura", "affiliation": "Chief Medical Officer for the defense health agency region Indo-Pacific" },
                  { "name": "Mark Vaught", "affiliation": "Mahi Pono" },
                  { "name": "Cal Chipchase", "affiliation": "Molokai Ranch" },
                  { "name": "Harold Edwards", "affiliation": "ITC Consultant/Contractor" }
                ]
              },
            "spontaneous_requests": [
                {
                  "request": "Chair Chang requested a motion to take C1 and C2 out of the agenda order.",
                  "motion": { 
                    "motion_proposal": "Take item C1 and item C2 and move up.",
                    "proposed_by": "HO",
                    "seconded_by": "Meyer",
                    "action": "Unanimously Approved",
                    "votes": {
                      "for": ["HO","BUCK","HANNAHS","KAGAWA-VIVIANI","KATAYAMA","MEYER","CHANG"],
                      "against": [],
                      "abstain": [],
                      "absent": []
                    }
                  }
                },
              ],
            
          }
    },
]

In [277]:
# Example function to match section titles to schema names
def normalize_section_title(section_title):
    section_title = section_title.strip().lower()
    
    if "approval" in section_title:
        return "APPROVAL OF MINUTES"
    elif "non-action" in section_title or "informational" in section_title:
        return "NON-ACTION ITEMS / INFORMATIONAL BRIEFINGS"
    elif "action" in section_title:
        return "ACTION ITEMS"
    elif "minutes" in section_title:
        return "INTRO"
    
    return None

# Function to match the section to the correct nested schema head
def get_matching_schema(section, schemas):
    # Normalize the section title for matching
    normalized_section = normalize_section_title(section)
    
    # Iterate through all schemas
    for schema in schemas:
        schema_header = schema.get("header", "")
        
        if normalized_section == schema_header:
            return schema_header
    
    return None


matched_schema = get_matching_schema(result["chunk_4"]["section"], schemas)
print(matched_schema)

ACTION ITEMS


In [278]:
# Loop through the result dictionary and add the matched schema to each section
def add_matched_schemas(result, schemas):
    for key, value in result.items():
        if "section" in value:  # Check if section exists in the value
            section_title = value["section"]
            matched_schema = get_matching_schema(section_title, schemas)
            result[key]["matched_schema"] = matched_schema  # Add new key-value pair

# Add matched schema name to each chunk in result
add_matched_schemas(result, schemas)

In [279]:
def create_prompt(section, schema):
    prompt = f"""
    please extract all information from the following text of meeting minutes and fill in the corresponding key:value pairs referencing the JSON schema example provided below. 
    
    - Do not remove "___...___" from text.
    - Please first read all text, for text subsections "questions/comments", "discussion", and "public testimony", Please include all text from these sections.
    - Return all of the schema structure, only include text from the input text.

    Schema:
    {schema}

    Text:
    {section}
    """
    return prompt.strip()

def process_section_with_llm(section_content, schema):
    prompt = create_prompt(section_content, schema)
    response = client.chat.completions.create(
        model="gpt-4o-2024-08-06", 
        messages=[
            {"role": "system", "content": "You are an AI assistant that uses text from meeting minutes and formats it as JSON according to the provided schema."},
            {"role": "user", "content": prompt}
        ],
        temperature=0,
        response_format={ "type": "json_object" },
    )
    output = response.choices[0].message.content
    return output

In [280]:
# Loop through each chunk in the result and process each section
def process_all_chunks(result, schemas):
    for key, value in result.items():
        if "section" in value and "matched_schema" in value:
            section_content = value["truncated_text"]  
            matched_schema = value["matched_schema"]  
            
            # Iterate through all schemas
            for schema in schemas:
                schema_header = schema.get("header", "")
                if matched_schema == schema_header:
                    schema = schema.get("schema", "")
                    break

            # Process the section content with the LLM
            processed_output = process_section_with_llm(section_content, schema)
            
            # Save the processed output in the result dictionary
            result[key]["processed_output"] = processed_output

process_all_chunks(result, schemas)

In [281]:
print(json.dumps(result["chunk_4"], indent=4))

{
    "section": "**B. ACTION ITEM**",
    "original_text": "**B. ACTION ITEM**\n\n2. Modification of Existing Delegation to the Chairperson, Dated December 19, 2023, to Approve Certain Well Construction and Pump Installation Permits by Limiting the Delegation to Certain Circumstances When the Well is in Compliance with the Hawai\u2018i Well Construction and Pump Installation Standards and for Pumps Less than 27 Gallons Per Minute and Delegation to the Deputy Director for Other Groundwater Permit-Related Approvals Including Acceptance of Applications for Initial Review, Permit Extensions, and Well Completion Reports and Certificates.\n\n**PRESENTATION OF SUBMITTAL:** Ryan Imata, Groundwater Regulation Branc# Chief\n\nStaff recommended revisions to the delegations to approve well and pump permits, set by the Commission at its December 19, 2023 meeting, after the Commission\u2019s directive to consult with the Departments of Water Supply.\n\n## QUESTIONS / COMMENTS\n\n**COMMISSIONER MIIK

In [282]:
print(result["chunk_4"]["processed_output"])


{
    "action_items": {
        "section": "B",
        "timestamp": "20240618 01:47:52",
        "submittals": [
            {
                "submittal": "B2",
                "title": "Modification of Existing Delegation to the Chairperson, Dated December 19, 2023, to Approve Certain ___...___ Initial Review, Permit Extensions, and Well Completion Reports and Certificates.",
                "timestamp": "20240618 01:47:52",
                "presentations": [
                    {
                        "presenter": {
                            "name": "Ryan Imata",
                            "affiliation": "Groundwater Regulation Branch Chief"
                        },
                        "external_resource": false,
                        "external_resource_location": "",
                        "comments": "Staff recommended revisions to the delegations to approve well and pump permits, set by the Commission at its December 19, 2023 meeting, after the Commission’s direct